# Level 4: Expert Techniques (Ensemble)

**Objective**: Build an ensemble model to push accuracy >93%.

**Method**: 
- Load **Level 2 Model** (ResNet50 Augmented)
- Load **Level 3 Model** (EfficientNet Fine-tuned)
- Perform **Soft Voting** (Average Probabilities) on the Test Set.

In [1]:
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def build_resnet():
    m = models.resnet50(pretrained=False)
    m.fc = nn.Linear(m.fc.in_features, 102)
    return m

def build_efficientnet():
    m = models.efficientnet_b0(pretrained=False)
    m.classifier[1] = nn.Linear(m.classifier[1].in_features, 102)
    return m

In [ ]:
class MergedFlowersDataset(Dataset):
    def __init__(self, image_files, labels, transform=None):
        self.image_files = image_files
        self.labels = labels
        self.transform = transform
        self.loader = datasets.folder.default_loader
    def __len__(self): return len(self.image_files)
    def __getitem__(self, idx):
        return self.transform(self.loader(self.image_files[idx])), self.labels[idx]

def get_test_loader():
    mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
    tf = transforms.Compose([
        transforms.Resize((256, 256)), transforms.CenterCrop(224),
        transforms.ToTensor(), transforms.Normalize(mean, std)
    ])
    
    all_samples, all_labels = [], []
    for s in ['train', 'val', 'test']:
        try:
            ds = datasets.Flowers102('./data', split=s, download=True)
            all_samples.extend(ds._image_files); all_labels.extend(ds._labels)
        except: pass
    
    idx = np.arange(len(all_samples))
    _, tmp = train_test_split(idx, test_size=0.2, stratify=all_labels, random_state=42)
    _, test_i = train_test_split(tmp, test_size=0.5, stratify=np.array(all_labels)[tmp], random_state=42)
    
    ds = MergedFlowersDataset([all_samples[i] for i in test_i], [all_labels[i] for i in test_i], tf)
    return DataLoader(ds, batch_size=32, shuffle=False, num_workers=0)

test_loader = get_test_loader()

100%|██████████| 345M/345M [00:22<00:00, 15.6MB/s] 
100%|██████████| 502/502 [00:00<00:00, 60.1kB/s]
100%|██████████| 15.0k/15.0k [00:00<00:00, 4.38MB/s]


In [ ]:
m1 = build_resnet().to(device)
m2 = build_efficientnet().to(device)

try:
    m1.load_state_dict(torch.load('models/level_2_augmented.pth'))
    m2.load_state_dict(torch.load('models/level_3_efficientnet.pth'))
    print("Models loaded successfully!")
except FileNotFoundError:
    print("Warning: Checkpoints not found. Ensure Level 2 and 3 notebooks have been run.")

c:\Users\sansk\PROJECTS\terafac\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\sansk\PROJECTS\terafac\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


## 1. Ensemble Evaluation

In [ ]:
m1.eval()
m2.eval()

correct_m1 = 0
correct_m2 = 0
correct_ensemble = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        out1 = m1(inputs)
        out2 = m2(inputs)
        
        prob1 = torch.softmax(out1, dim=1)
        prob2 = torch.softmax(out2, dim=1)
        
        avg_prob = (prob1 + prob2) / 2.0
        
        _, pred1 = torch.max(prob1, 1)
        _, pred2 = torch.max(prob2, 1)
        _, pred_ens = torch.max(avg_prob, 1)
        
        total += labels.size(0)
        correct_m1 += (pred1 == labels).sum().item()
        correct_m2 += (pred2 == labels).sum().item()
        correct_ensemble += (pred_ens == labels).sum().item()

print(f"ResNet50 Acc:      {correct_m1/total:.4f}")
print(f"EfficientNet Acc:  {correct_m2/total:.4f}")
print(f"Ensemble Acc:      {correct_ensemble/total:.4f}")

ResNet50 Acc:      0.0122
EfficientNet Acc:  0.0049
Ensemble Acc:      0.0122


# Research Report Components

### 1. Model Architecture
- **Model A**: ResNet50 (Pretrained on ImageNet). Fixed backbone, trained FC head. Enhanced with random rotation/crops.
- **Model B**: EfficientNet-B0. Fine-tuned end-to-end. Uses Scaled Dot-Product Attention implicitly via architecture design (SE blocks).

### 2. Ensemble Strategy
- employed **Soft Voting**, averaging the softmax probability distributions of both models. This captures the confidence of each model rather than just the hard class label, often leading to better calibration and accuracy.

### 3. Key Findings
- EfficientNet generally outperforms ResNet50 on fine-grained tasks due to better feature resolution handling.
- Examples where Ensemble succeeded: [Insert qualitative analysis]
- Examples where Ensemble failed: [Insert failure cases]